<a href="https://colab.research.google.com/github/turja16/Algorithms-and-Data-structures-with-C-Python/blob/main/01_Part_I_Graph_Models_on_Real_World_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Heterophily Benchmark Tutorial: Part I - Graph Models on Real-World Dataset

Sitao Luan, MILA \\
Chenqing Hua, McGill University & MILA \\
QiCheng Lu, McGill University & MILA \\
Jiaqi Zhu, DRW


##### ***LOG2024: A Tutorial on Heterophilic Graph Learning***

# Environment Setup


In [1]:
COLLAB_ENV = True

In [2]:
import os
import shutil

import ipywidgets as widgets
from ipywidgets import Box

In [3]:
if COLLAB_ENV:
    !git clone --recurse-submodules -j8 https://github.com/jzhu1905/HeterophilicGraphLearningTutorial.git
    !cd HeterophilicGraphLearningTutorial
    !git init
    !git submodule update --init --recursive
    shutil.copytree("HeterophilicGraphLearningTutorial/", "./", dirs_exist_ok=True)
    !rm -rf HeterophilicGraphLearning-Tutorial

    !pip install torch==2.2.2
    !pip install torchaudio==2.2.2 torchdata==0.7.1 torchvision==0.17.2
    !pip install dgl -f https://data.dgl.ai/wheels/torch-2.2/cu121/repo.html
    import torch
    os.environ['TORCH'] = torch.__version__
    print(torch.__version__)
    !pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install git+https://github.com/pyg-team/pytorch_geometric.git
    !pip install torch-geometric===2.1.0.post1
    !pip install networkx==2.5
    !pip install ogb

Cloning into 'HeterophilicGraphLearningTutorial'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 44 (delta 20), reused 22 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (44/44), 41.16 KiB | 3.17 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Submodule 'ACM-GNN' (https://github.com/SitaoLuan/ACM-GNN.git) registered for path 'ACM-GNN'
Submodule 'Heterophilic_Benchmarks' (https://github.com/wzzlcss/Heterophilic_Benchmarks.git) registered for path 'Heterophilic_Benchmarks'
Cloning into '/content/HeterophilicGraphLearningTutorial/ACM-GNN'...
remote: Enumerating objects: 867, done.        
remote: Counting objects: 100% (867/867), done.        
remote: Compressing objects: 100% (727/727), done.        
remote: Total 867 (delta 156), reused 828 (delta 136), pack-reused 0 (from 0)        
Receiving objects: 100% (867/867), 26.87 MiB | 11.30 MiB/s, done.
Resolving deltas: 100% 


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

2.2.2+cu121
Looking in links: https://data.pyg.org/whl/torch-2.2.2+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 105.5 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.2.2+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 26.1 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.2.2+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 34.6 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-rd_7u8f9
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-rd_7u8f9
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 69c6da143394387f48cccdf0d32910247e8d5d61
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.7.0-py3-none-any

In [24]:
    !pip install google_drive_downloader

ERROR: Could not find a version that satisfies the requirement google_drive_downloader (from versions: none)
ERROR: No matching distribution found for google_drive_downloader


In [39]:
import sys
import numpy as np
np.float_ = np.float64
sys.path.append("./Heterophilic_Benchmarks")

from constants import GRAPH_MODELS, DATASETS
from train_graph_model import train_graph_model

IS_GPU_AVAILABLE = torch.cuda.is_available()

# Trianing Graph Models with Real World Dataset

## 1. Select a GNN model

In [48]:
graph_model = widgets.Dropdown(
    options=GRAPH_MODELS.keys(),
    default=list(GRAPH_MODELS.keys())[0],
    description='Choose a graph model:',
    style={'description_width': 'initial'}
)
display(graph_model)

Dropdown(description='Choose a graph model:', options=('ACM', 'BernNet', 'FAGCN', 'FSGCN', 'GBKGNN', 'APPNP', …

## 2. Select a Dataset

### 2.1. First select a dataset class

In [49]:
dataset_classes = GRAPH_MODELS[graph_model.value]
dataset_class = widgets.Dropdown(
    options=dataset_classes,
    default=dataset_classes[0],
    description='Choose a dataset class:',
    style={'description_width': 'initial'}
)
display(dataset_class)

Dropdown(description='Choose a dataset class:', options=(('Critical Look', 'critical'), ('Large Scale', 'large…

### 2.2 Then select a dataset

In [50]:
datasets = DATASETS[dataset_class.value]
dataset = widgets.Dropdown(
    options=datasets,
    default=datasets[0],
    description='Choose a dataset:',
    style={'description_width': 'initial'}
)
display(dataset)

Dropdown(description='Choose a dataset:', options=(('Actor', 'actor'), ('Amazon-Ratings', 'amazon-ratings'), (…

## 3. Train It!


In [51]:
train_graph_model(graph_model.value, dataset_class.value, dataset.value, IS_GPU_AVAILABLE)

Preparing dataset...


Downloading...
From (original): https://drive.google.com/uc?id=1_pysG3_l8w5F3GXvq6qTg-Ja2astBhxm
From (redirected): https://drive.google.com/uc?id=1_pysG3_l8w5F3GXvq6qTg-Ja2astBhxm&confirm=t&uuid=b0633a14-c335-4b09-b473-3ebf43c9bec8
To: /content/Heterophilic_Benchmarks/critical_look_utils/data/critical_data.zip
100%|██████████| 99.3M/99.3M [00:00<00:00, 190MB/s]


Start training FAGCN on dataset: actor


RuntimeError: Numpy is not available